# Reuse Model

In [ ]:
import numpy as np
import pickle
import cv2
import os
import matplotlib.pyplot as plt
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
# Load model
#filename = 'plant_disease_classification_model.pkl'
#model = pickle.load(open(filename, 'rb'))

from keras.models import load_model

model = load_model('Trained_files/apple/plant_disease_classification_model_apple.h5')

# Load labels
filename = 'Trained_files/apple/plant_disease_label_transform_apple.pkl'
image_labels = pickle.load(open(filename, 'rb'))

In [ ]:
# Dimension of resized image
IMAGE_RESIZE = tuple((256, 256))

# converting images to array
def convert_image_to_array(image_directory):
    try:
        image = cv2.imread(image_directory)
        if image is not None:
            image = cv2.resize(image, IMAGE_RESIZE)   
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

def predict_disease(image_path):
    image_array = convert_image_to_array(image_path)
    np_image = np.array(image_array, dtype=np.float16) / 225.0
    np_image = np.expand_dims(np_image,0)
    plt.imshow(plt.imread(image_path))
    #result = model.predict_classes(np_image)
    result=np.argmax(model.predict(np_image), axis=-1)
    print((image_labels.classes_[result][0]))

In [ ]:
predict_disease('Random_images/001.jpg')

In [ ]:
#________________________________________________________________________

In [1]:

import cv2 
from flask import Flask, render_template, Response, flash, request, redirect, url_for, render_template
import urllib.request
from werkzeug.utils import secure_filename
from keras.models import Sequential
import os
import numpy as np
import pickle
from os import listdir
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import load_model

app = Flask(__name__)

UPLOAD_FOLDER = 'df_img/'
model = load_model('Trained_files/1st_training/plant_disease_classification_model.h5')

# Load labels
filename = 'Trained_files/1st_training/plant_disease_label_transform.pkl'
image_labels = pickle.load(open(filename, 'rb'))
# Dimension of resized image
IMAGE_RESIZE = tuple((224, 224))
 
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
 
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg']) # specifing image formats that we accept.

# create a method to check whether the uploaded file is aloowcated.
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, IMAGE_RESIZE)   
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None   

@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')


@app.route('/', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash('No image selected for uploading')
        return redirect(request.url)
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        # image processing to predict 
        image_array = convert_image_to_array(filepath)
        np_image = np.array(image_array, dtype=np.float16) / 225.0
        np_image = np.expand_dims(np_image,0)
        plt.imshow(plt.imread(filepath))
        #result = model.predict_classes(np_image)
        result=np.argmax(model.predict(np_image), axis=-1)
        flash('Image successfully uploaded and displayed below')
        flash((image_labels.classes_[result][0])) # flashing the disease
        return render_template('index.html', filename=filename)
    else:
        flash('Allowed image types are - png, jpg, jpeg')
        return redirect(request.url)
 
@app.route('/display/<filename>')
def display_image(filename):
    print('display_image filename: ' + filename)
    return redirect(url_for(filename ='df_img/' + filename), code=301)
 
if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
